In [4]:
participant_id = "P4"

In [5]:
%load_ext autoreload
%autoreload 2
from hireverse.utils.utils import *

video_file_path = os.path.join(BASE_DIR, "data/raw/videos", f"{participant_id}.avi")
print(video_file_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/Users/bassel27/personal_projects/hireverse/data/raw/videos/P4.avi


In [6]:
from hireverse.utils.face_analyzer import FaceAnalyzer
from hireverse.schemas.frame import Frame
face_analyzer = FaceAnalyzer()
frames = face_analyzer.get_video_frames(participant_id="randomId",video_path= video_file_path, target_fps=20)

In [7]:
filtered_frames: List[Frame] = []
for frame in frames:
    detected_faces_landmarks = face_analyzer.process_image_results(frame.image)
    frame.facial_landmarks_obj = face_analyzer.get_largest_face_landmarks_obj(frame.image, detected_faces_landmarks)
    if frame.facial_landmarks_obj:
        frame.facial_landmarks = frame.facial_landmarks_obj.landmark
        filtered_frames.append(frame)

frames = filtered_frames 

W0000 00:00:1745836356.068528 1568704 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


In [8]:
for frame in frames:
    frame.resize(new_width=640)

In [9]:
for frame in frames:
    frame.align_face_with_mediapipe_landmarks()

In [10]:
for frame in frames:
    if frame.facial_landmarks:
        frame.face = face_analyzer.get_face_coordinates(frame.facial_landmarks, frame.image)
        x, y, w, h = frame.face
        frame.crop_frame(x, y, x + w, y+h)

In [11]:
import cv2

for frame in frames:
    frame.image =  cv2.cvtColor(frame.image, cv2.COLOR_BGR2GRAY)

In [12]:
for frame in frames:
    frame.image = frame.image.astype('float32') / 255.0

In [13]:
for frame in frames:
    frame.resize(new_width=640, new_height=640)

In [ ]:
import numpy as np


for frame in frames:
    frame.reset_drawable_image()
    # frame.draw_face_border()
    # frame.draw_facial_landmarks()
    # if frame.facial_landmarks:
    #     frame.draw_circle_at_facial_landmark(frame.facial_landmarks[10], frame.facial_landmarks[152])
    frame.display()

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
